In [ ]:
accuracy_matrix = np.zeros((20, 20))

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image  # Import PIL Image

# Custom Dataset for datasets with or without labels
class CIFARDataset(Dataset):
    def __init__(self, data_array, labels_array=None, transform=None):
        self.data = data_array  # NumPy array of images
        self.labels = labels_array  # NumPy array of labels or None
        self.transform = transform

    def __len__(self):
        return len(self.data)  # Use len instead of size(0)

    def __getitem__(self, idx):
        image = self.data[idx]
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            # Ensure label is a scalar
            if isinstance(label, np.ndarray):
                label = int(label)  # Convert NumPy array to scalar
            elif torch.is_tensor(label):
                label = label.item()
            return image, label
        else:
            return image

# Function to load dataset from .tar.pth files
def load_dataset(file_path):
    # Load the .tar.pth file (contains 'data' and 'targets')
    data_dict = torch.load(file_path)
    data = data_dict['data']  # NumPy array of images
    labels = data_dict.get('targets', None)  # Use 'targets' instead of 'labels'
    return data, labels

# Functions to extract features
def extract_features(dataloader, feature_extractor, device):
    features = []
    labels = []

    with torch.no_grad():
        for inputs, label in dataloader:
            inputs = inputs.to(device)
            outputs = feature_extractor(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs.cpu())
            labels.extend(label)  # Use extend instead of append

    features = torch.cat(features, dim=0)
    labels = torch.tensor(labels)  # Convert list of labels to tensor
    return features, labels

def extract_features_unlabeled(dataloader, feature_extractor, device):
    features = []

    with torch.no_grad():
        for inputs in dataloader:
            if isinstance(inputs, list) or isinstance(inputs, tuple):
                inputs = inputs[0]  # Extract inputs from list/tuple
            inputs = inputs.to(device)
            outputs = feature_extractor(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs.cpu())

    features = torch.cat(features, dim=0)
    return features

# Classifier model
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes=10):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

# Main training and evaluation function
def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Data transformations
    data_transforms = transforms.Compose([
        # Removed transforms.ToPILImage() since images are already PIL Images
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    # Feature extractor
    feature_extractor = models.resnet152(pretrained=True)
    feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])
    feature_extractor = feature_extractor.to(device)
    feature_extractor.eval()


    # Paths to datasets
    dataset_folder = 'dataset'
    part_one_dataset = os.path.join(dataset_folder, 'part_one_dataset')
    part_two_dataset = os.path.join(dataset_folder, 'part_two_dataset')

    # Load D1 (dataset index 1 in part_one_dataset)
    D1_train_data_path = os.path.join(part_one_dataset, 'train_data', '1_train_data.tar.pth')
    D1_data, D1_labels = load_dataset(D1_train_data_path)
    D1_dataset = CIFARDataset(D1_data, D1_labels, transform=data_transforms)
    D1_dataloader = DataLoader(D1_dataset, batch_size=64, shuffle=False)
    D1_features, D1_labels = extract_features(D1_dataloader, feature_extractor, device)

    # Initialize model f1
    input_dim = D1_features.size(1)
    f_prev = Classifier(input_dim, num_classes=10).to(device)

    # Train f1 on D1
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(f_prev.parameters(), lr=0.01, momentum=0.9)

    num_epochs = 20
    dataset = torch.utils.data.TensorDataset(D1_features, D1_labels)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

    print("Training model f1 on D1")

    # Loss components
    criterion_ce = nn.CrossEntropyLoss()  # For pseudo-labels
    temperature = 1.0  # Distillation temperature
    lambda_distill = 0.1  # Weight for distillation loss
    lambda_reg = 0.01  # Weight for L2 regularization

    theta_prev = {name: param.detach().clone() for name, param in f_prev.named_parameters()}

    for epoch in range(num_epochs):
        f_prev.train()
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Current model predictions
            outputs_current = f_prev(inputs)
            loss_ce = criterion_ce(outputs_current, labels)

            # Previous model predictions (soft targets)
            with torch.no_grad():
                outputs_prev = f_prev(inputs)
            soft_targets_prev = torch.softmax(outputs_prev / temperature, dim=1)
            soft_targets_current = torch.log_softmax(outputs_current / temperature, dim=1)
            loss_distill = -torch.sum(soft_targets_prev * soft_targets_current) / inputs.size(0)

            # L2 regularization
            l2_reg = 0.0
            for name, param in f_prev.named_parameters():
                l2_reg += torch.norm(param - theta_prev[name].to(device)) ** 2

            # Combined loss
            loss = loss_ce + lambda_distill * loss_distill + lambda_reg * l2_reg

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader):.4f}')

    # Initialize accuracy matrix
    # accuracy_matrix = np.zeros((20, 20))

    # Loop over datasets D2 to D10
    for i in range(1, 11):
        print(f'\nProcessing Dataset D{i}')

        if i <= 10:
            # Datasets D2 to D10 in part_one_dataset
            train_data_path = os.path.join(part_one_dataset, 'train_data', f'{i}_train_data.tar.pth')
        else:
            # Datasets D11 to D20 in part_two_dataset
            idx = i - 10  # Since files are numbered from 1 to 10
            train_data_path = os.path.join(part_two_dataset, 'train_data', f'{idx}_train_data.tar.pth')

        # Load Di
        Di_data, _ = load_dataset(train_data_path)
        Di_dataset = CIFARDataset(Di_data, transform=data_transforms)
        Di_dataloader = DataLoader(Di_dataset, batch_size=32, shuffle=False)
        Di_features = extract_features_unlabeled(Di_dataloader, feature_extractor, device)

        # Predict labels using f_prev
        f_prev.eval()
        with torch.no_grad():
            outputs = f_prev(Di_features.to(device))
            _, predicted_labels = torch.max(outputs, 1)
            predicted_labels = predicted_labels.cpu()

        # Prepare dataset with pseudo-labels
        dataset = torch.utils.data.TensorDataset(Di_features, predicted_labels)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

        # Initialize current model fi
        fi = Classifier(input_dim, num_classes=10).to(device)
        fi.load_state_dict(f_prev.state_dict())

# Store the parameters of the previous model (f_prev)
        theta_prev = {name: param.detach().clone() for name, param in f_prev.named_parameters()}


        # Regularization parameter
        lambda_reg = 1.0 if i <= 10 else 0.1  # Adjusted for potential distribution shift

        # Store previous parameters
        for name, param in f_prev.named_parameters():
            theta_prev[name] = param.detach().clone()


                # Initialize optimizer and scheduler
        optimizer = optim.Adam(fi.parameters(), lr=0.001, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

        # Training fi on Di with pseudo-labels
        print(f"Training model f{i} on D{i}")
        for epoch in range(num_epochs):
            fi.train()
            running_loss = 0.0
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Predictions from f_prev (teacher) for distillation
                with torch.no_grad():
                    outputs_prev = f_prev(inputs)

                # Predictions from fi (student)
                outputs_current = fi(inputs)

                # Cross-entropy loss
                loss_ce = criterion(outputs_current, labels)

                # Distillation loss
                soft_targets_prev = torch.softmax(outputs_prev / temperature, dim=1)
                soft_targets_current = torch.log_softmax(outputs_current / temperature, dim=1)
                loss_distill = -torch.sum(soft_targets_prev * soft_targets_current) / inputs.size(0)

                # Regularization loss
                l2_reg = 0.0
                for name, param in fi.named_parameters():
                    l2_reg += torch.norm(param - theta_prev[name].to(device)) ** 2

                # Combined loss
                loss = loss_ce + lambda_distill * loss_distill + lambda_reg * l2_reg

                # Backpropagation
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            # Update learning rate
            scheduler.step()

            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader):.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Evaluation on D_hat datasets
        for j in range(1, i+1):
            if j <= 10:
                # D_hat datasets in part_one_dataset
                eval_data_path = os.path.join(part_one_dataset, 'eval_data', f'{j}_eval_data.tar.pth')
                D_hat_data, D_hat_labels = load_dataset(eval_data_path)
            else:
                # D_hat datasets in part_two_dataset
                idx = j - 10
                eval_data_path = os.path.join(part_two_dataset, 'eval_data', f'{idx}_eval_data.tar.pth')
                D_hat_data, D_hat_labels = load_dataset(eval_data_path)

            D_hat_dataset = CIFARDataset(D_hat_data, D_hat_labels, transform=data_transforms)
            D_hat_dataloader = DataLoader(D_hat_dataset, batch_size=64, shuffle=False)
            D_hat_features, D_hat_labels = extract_features(D_hat_dataloader, feature_extractor, device)

            fi.eval()
            with torch.no_grad():
                outputs = fi(D_hat_features.to(device))
                _, predicted = torch.max(outputs, 1)
                correct = (predicted.cpu() == D_hat_labels).sum().item()
                total = D_hat_labels.size(0)
                accuracy = correct / total
                print(f'Model f{i} Accuracy on D_hat{j}: {accuracy*100:.2f}%')

                # Store accuracy in matrix
                accuracy_matrix[i-1][j-1] = accuracy * 100

        # Update f_prev to fi
        f_prev = fi

    # Display accuracy matrix
    print('\nAccuracy Matrix:')
    for i in range(20):
        accuracies = ['{:.2f}'.format(acc) for acc in accuracy_matrix[i][:i+1]]
        print(f'Model f{i+1}: ' + ', '.join(accuracies))

if __name__ == '__main__':
    main()
